In [46]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
import preproccessing as prep
from evaluation import show_scenarios_evaluation
from regime_switching_brownian_motion import RegimeSwitching
from independent_lognormal_model import IndependentLogNormal
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
def convert_to_transition_matrix(p00,p10):
     k =2
     transition_matrix = np.zeros((k,k))
     transition_matrix[0,0] = p00
     transition_matrix[1,0] = p10
     transition_matrix[:,1] = 1 - transition_matrix[:,0]
     return transition_matrix


In [19]:
world = pd.read_excel('msci_world_usd_30.10.23.xlsx',index_col='Date', parse_dates=True)
world = world.squeeze()
world_returns = prep.log_returns(world)
world_returns.info()

<class 'pandas.core.series.Series'>
DatetimeIndex: 645 entries, 1970-01-30 to 2023-09-29
Series name: WORLD Standard (Large+Mid Cap)
Non-Null Count  Dtype  
--------------  -----  
645 non-null    float64
dtypes: float64(1)
memory usage: 10.1 KB


In [89]:
periods = [
    ('1970-11-01', '2023-10-30'),
    ('1973-11-01', '2023-10-30'),
    ('1973-11-01', '2019-12-31'),
    ('1983-11-01', '2023-10-30'),
    ('1993-11-01', '2023-10-30'),
    ('1994-01-01', '2023-10-30'),
    ('1999-01-01', '2023-10-30'),
    ('2003-11-01', '2023-10-30'),    
    ('2013-11-01', '2023-10-30'),
]

prep.asset_stats(world_returns,periods)

,Annualised Mean,Annualised Sigma,Skew,Kurtosis,Sharp_ratio,Max drowdawn
"(1970-11-01, 2023-10-30)",0.086692,0.149740,-0.741006,2.083054,1.311509,-0.540291
"(1973-11-01, 2023-10-30)",0.083414,0.152316,-0.734475,1.982212,1.214788,-0.540291
"(1973-11-01, 2019-12-31)",0.084773,0.148525,-0.775601,2.290275,1.277495,-0.540291
"(1983-11-01, 2023-10-30)",0.087665,0.153639,-0.821811,2.135338,1.300174,-0.540291
"(1993-11-01, 2023-10-30)",0.069209,0.153336,-0.846681,1.913494,0.885784,-0.540291
"(1994-01-01, 2023-10-30)",0.069964,0.153121,-0.853065,1.959404,0.904120,-0.540291
"(1999-01-01, 2023-10-30)",0.054665,0.157324,-0.775767,1.730128,0.543091,-0.540291
"(2003-11-01, 2023-10-30)",0.073676,0.155547,-0.923110,2.514783,0.972673,-0.540291
"(2013-11-01, 2023-10-30)",0.076207,0.147438,-0.532838,1.085640,1.085649,-0.254242


my chosen params
mu1 = 1.35
mu2 = -0.11
sig1 = 0.07
sig2 = 0.33
p00 = 92.38
p20 = 8.74

In [92]:
data = world_returns
n_regimes = 2
params = []

for period in periods:
    model = sm.tsa.MarkovRegression(
    data[period[0]:period[1]],
    k_regimes=n_regimes,
    switching_variance=True,
    freq=None
    )
    result = model.fit()
    for k in range(n_regimes):
        result.params[f'sigma2[{k}]'] = np.sqrt(result.params[f'sigma2[{k}]'])
    
    params.append(np.round(result.params * 100,2))

params = pd.DataFrame(data=params, index=periods)
params

c:\Users\matsz\miniconda3\envs\portfolio-optimisation\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency BM will be used.
  self._init_dates(dates, freq)
c:\Users\matsz\miniconda3\envs\portfolio-optimisation\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency BM will be used.
  self._init_dates(dates, freq)
c:\Users\matsz\miniconda3\envs\portfolio-optimisation\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency BM will be used.
  self._init_dates(dates, freq)
c:\Users\matsz\miniconda3\envs\portfolio-optimisation\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency BM will be used.
  self._init_dates(dates, freq)
c:\Users\matsz\miniconda3\envs\portfolio-optimisation\lib\si

,p[0->0],p[1->0],const[0],const[1],sigma2[0],sigma2[1]
"(1970-11-01, 2023-10-30)",92.89,9.55,1.36,-0.13,2.63,5.75
"(1973-11-01, 2023-10-30)",92.36,8.90,1.37,-0.11,2.60,5.75
"(1973-11-01, 2019-12-31)",93.15,8.74,1.35,-0.13,2.59,5.67
"(1983-11-01, 2023-10-30)",91.88,8.38,1.44,-0.00,2.46,5.71
"(1993-11-01, 2023-10-30)",94.20,5.34,1.35,-0.18,2.37,5.66
"(1994-01-01, 2023-10-30)",94.50,5.50,1.34,-0.20,2.40,5.70
"(1999-01-01, 2023-10-30)",94.75,4.42,1.35,-0.31,2.33,5.68
"(2003-11-01, 2023-10-30)",94.66,6.41,1.30,-0.21,2.29,6.05
"(2013-11-01, 2023-10-30)",72.29,9.99,1.76,0.24,0.81,4.85


In [96]:
periods = [
    ('1970-01-01', '1994-12-31'),
    ('1980-01-01', '2004-12-31'),
    ('1990-01-01', '2014-12-31'),
    ('1998-01-01', '2023-12-31'),
]
data = world_returns
n_regimes = 2
params = []

for period in periods:
    model = sm.tsa.MarkovRegression(
    data[period[0]:period[1]],
    k_regimes=n_regimes,
    switching_variance=True,
    freq=None
    )
    result = model.fit()
    for k in range(n_regimes):
        result.params[f'sigma2[{k}]'] = np.sqrt(result.params[f'sigma2[{k}]'])
    
    params.append(np.round(result.params * 100,2))

params = pd.DataFrame(data=params, index=periods)
params

c:\Users\matsz\miniconda3\envs\portfolio-optimisation\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency BM will be used.
  self._init_dates(dates, freq)
c:\Users\matsz\miniconda3\envs\portfolio-optimisation\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency BM will be used.
  self._init_dates(dates, freq)
c:\Users\matsz\miniconda3\envs\portfolio-optimisation\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency BM will be used.
  self._init_dates(dates, freq)
c:\Users\matsz\miniconda3\envs\portfolio-optimisation\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency BM will be used.
  self._init_dates(dates, freq)


,p[0->0],p[1->0],const[0],const[1],sigma2[0],sigma2[1]
"(1970-01-01, 1994-12-31)",93.70,24.51,1.28,-1.02,3.25,6.39
"(1980-01-01, 2004-12-31)",92.41,14.53,1.74,-0.74,3.11,5.49
"(1990-01-01, 2014-12-31)",97.15,2.90,1.28,-0.27,2.59,5.61
"(1998-01-01, 2023-12-31)",94.10,4.86,1.40,-0.24,2.33,5.73


In [ ]:
t_m = convert_to_transition_matrix(0.924,0.087)
initial_values=100
current_regime=0
dt=1
n_scenarios=2000
n_steps= 300
hist=world_returns

def show_RSLN_scenarios_evaluation(mu1, mu2, sigma1, sigma2, p00,p10, initial_values, current_regime, dt, n_scenarios, n_steps, hist):
      
    transition_matrix = convert_to_transition_matrix(p00/100,p10/100)

    model = RegimeSwitching(
    models=[IndependentLogNormal(mu=mu1/100,sigma=sigma1/100),
            IndependentLogNormal(mu=mu2/100,sigma=sigma2/100)],
    probs=transition_matrix
    )

    scenarios = model.scenarios(initial_values,current_regime,dt,n_steps, n_scenarios)
    scenarios = pd.DataFrame(data=scenarios, index=range(n_scenarios), columns=range(n_steps+1))

    show_scenarios_evaluation(scenarios,hist)



interact_manual(show_RSLN_scenarios_evaluation,         
         mu1=(0.1,5,0.01),
         mu2=(-3,3,0.01),
         sigma1=(1,7, 0.1),
         sigma2=(1,7, 0.1),
         p00 = (1,99,0.1),
         p10 = (1,99,0.1),
         initial_values=100,
         current_regime=0,
         dt=1,
         n_scenarios=2000,
         n_steps= 300,
         hist=fixed(world_returns)
         )




interactive(children=(FloatSlider(value=2.5500000000000003, description='mu1', max=5.0, min=0.1, step=0.01), F…

<function __main__.show_RSLN_scenarios_evaluation(mu1, mu2, sigma1, sigma2, p00, p10, initial_values, current_regime, dt, n_scenarios, n_steps, hist)>

In [42]:
stat1 = np.percentile(scenarios.iloc[:,-1]/100,[10,25,50])


In [44]:
stat2 = np.percentile(scenarios.iloc[:,-1]/100,[10,25,50])

In [68]:
print(100000/stat1[1])
print(100000/stat2[1])
print(100000/2.22)

21083.16731722123
22380.966797853118
45045.045045045044



12	36	60	120	240	300
10	0.932668	0.921778	0.966735	1.176694	1.978558	2.658478
15	0.972179	1.002762	1.066503	1.354019	2.428522	3.322188
25	1.024112	1.120036	1.227752	1.630383	3.205819	4.468082
50	1.127523	1.345847	1.581214	2.370384	5.175782	7.800501
75	1.225537	1.605543	1.979596	3.274284	8.447888	13.676793
85	1.278267	1.757236	2.254466	3.953372	11.094023	18.573161
90	1.313597	1.863894	2.455982	4.499387	13.176372	22.261616

In [99]:
import gbm_gibbs as gibs
sampler = gibs.GibsSamplerGBM(world_returns['2003-01-01':'2023-07-31'],0.0064)
sampler.run()
mu = sampler.mu_dist
sig = sampler.sigma_dist
print(pd.Series(mu*12, name='mu').describe())
print(pd.Series(sig * np.sqrt(12), name='sigma').describe())


count    10001.000000
mean         0.084799
std          0.001215
min          0.000000
25%          0.084690
50%          0.084816
75%          0.084945
max          0.085589
Name: mu, dtype: float64
count    10001.000000
mean         0.218088
std          0.005741
min          0.000000
25%          0.214752
50%          0.217879
75%          0.221268
max          0.240232
Name: sigma, dtype: float64


In [100]:
sampler = gibs.GibsSamplerGBM(world_returns['1970-01-01':'2023-07-31'],0.0064)
sampler.run()
mu = sampler.mu_dist
sig = sampler.sigma_dist
print(pd.Series(mu*12, name='mu').describe())
print(pd.Series(sig * np.sqrt(12), name='sigma').describe())

count    10001.000000
mean         0.084969
std          0.001204
min          0.000000
25%          0.084941
50%          0.084987
75%          0.085033
max          0.085269
Name: mu, dtype: float64
count    10001.000000
mean         0.212261
std          0.004116
min          0.000000
25%          0.210283
50%          0.212197
75%          0.214261
max          0.223748
Name: sigma, dtype: float64


In [75]:
from typing import Any
from scipy.stats import norm

class XPdf:
    
    def __init__(self, mu, sigma) -> None:
        self.mu = mu
        self.sigma = sigma

    def __call__(self, x):
        return norm.pdf((x-self.mu)/self.sigma)
    

xpdf = XPdf(0.08,0.15)
xpdf(0.15)


0.357783429167969

In [111]:
p00 = 0.95
p10 = 0.08
mu1 = 0.1
mu2 = -0.015
sigma1 = 0.08
sigma2 = 0.19
transition_matrix = convert_to_transition_matrix(p00,p10)
xpdf1 = XPdf(mu1, sigma1)
xpdf2 = XPdf(mu2, sigma2)

def initial_state(xpdf1, xpdf2, t_m, x):
    """
    t_m : transition matrix 
    """
    pi1 = t_m[1,0]/(t_m[0,1]+t_m[1,0])
    pi2 = t_m[0,1]/(t_m[0,1]+t_m[1,0])

    f1 = (pi1*t_m[0,0] + pi2*t_m[1,0]) * xpdf1(x)
    f2 = (pi1*t_m[0,1] + pi2*t_m[1,1]) * xpdf2(x)
    fy = f1 + f2

    #probability state 1 given parameters
    pp1 = f1/fy
    pp2 = f2/fy

    return pi1, pi2, f1, f2, fy, pp1, pp2

initial_state(xpdf1,xpdf2,transition_matrix, 0.09)

x = [0.09, 0.13, -0.03]


""" 
px = prob x given pt1, pt-1 1 and 2 + pt2, pt-1 1 and 2
py = f(pt1=1,pt-1=1)+f(pt1=1,pt-1=2)+f(pt1=2,pt-1=1)+f(pt1=2,pt-1=2)
"""


(0.6153846153846151,
 0.3846153846153848,
 0.24359242273523035,
 0.131710254689736,
 0.37530267742496637,
 0.6490559150991705,
 0.3509440849008294)